# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 6 2023 8:30AM,254399,10,WT5511 Out,Eywa Pharma Inc.,Released
1,Jan 5 2023 3:08PM,254398,21,OTM100959667,"NBTY Global, Inc.",Released
2,Jan 5 2023 3:01PM,254397,21,OTM100959739,"NBTY Global, Inc.",Released
3,Jan 5 2023 2:59PM,254396,21,OTM100959666,"NBTY Global, Inc.",Released
4,Jan 5 2023 2:58PM,254395,20,8993874,"Emersa Waterbox, LLC",Released
5,Jan 5 2023 2:53PM,254394,21,OTM100959665,"NBTY Global, Inc.",Released
6,Jan 5 2023 2:50PM,254393,22,637558,"NBTY Global, Inc.",Released
7,Jan 5 2023 2:49PM,254392,10,MSP217259,"Methapharm, Inc.",Released
8,Jan 5 2023 2:49PM,254392,10,MSP217260,"Methapharm, Inc.",Released
9,Jan 5 2023 2:49PM,254392,10,MSP217261,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,254395,Released,1
41,254396,Released,1
42,254397,Released,1
43,254398,Released,1
44,254399,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254395,NaN,NaN,1.0
254396,NaN,NaN,1.0
254397,NaN,NaN,1.0
254398,NaN,NaN,1.0
254399,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254302,0.0,1.0,1.0
254307,0.0,3.0,3.0
254312,24.0,11.0,1.0
254319,0.0,0.0,50.0
254327,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254302,0,1,1
254307,0,3,3
254312,24,11,1
254319,0,0,50
254327,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254302,0,1,1
1,254307,0,3,3
2,254312,24,11,1
3,254319,0,0,50
4,254327,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254302,,1,1
1,254307,,3,3
2,254312,24,11,1
3,254319,,,50
4,254327,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 6 2023 8:30AM,254399,10,Eywa Pharma Inc.
1,Jan 5 2023 3:08PM,254398,21,"NBTY Global, Inc."
2,Jan 5 2023 3:01PM,254397,21,"NBTY Global, Inc."
3,Jan 5 2023 2:59PM,254396,21,"NBTY Global, Inc."
4,Jan 5 2023 2:58PM,254395,20,"Emersa Waterbox, LLC"
5,Jan 5 2023 2:53PM,254394,21,"NBTY Global, Inc."
6,Jan 5 2023 2:50PM,254393,22,"NBTY Global, Inc."
7,Jan 5 2023 2:49PM,254392,10,"Methapharm, Inc."
25,Jan 5 2023 2:45PM,254391,21,"NBTY Global, Inc."
26,Jan 5 2023 2:45PM,254389,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 6 2023 8:30AM,254399,10,Eywa Pharma Inc.,,,1
1,Jan 5 2023 3:08PM,254398,21,"NBTY Global, Inc.",,,1
2,Jan 5 2023 3:01PM,254397,21,"NBTY Global, Inc.",,,1
3,Jan 5 2023 2:59PM,254396,21,"NBTY Global, Inc.",,,1
4,Jan 5 2023 2:58PM,254395,20,"Emersa Waterbox, LLC",,,1
5,Jan 5 2023 2:53PM,254394,21,"NBTY Global, Inc.",,,1
6,Jan 5 2023 2:50PM,254393,22,"NBTY Global, Inc.",,,1
7,Jan 5 2023 2:49PM,254392,10,"Methapharm, Inc.",,,18
8,Jan 5 2023 2:45PM,254391,21,"NBTY Global, Inc.",,,1
9,Jan 5 2023 2:45PM,254389,19,"Emersa Waterbox, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:30AM,254399,10,Eywa Pharma Inc.,1,,
1,Jan 5 2023 3:08PM,254398,21,"NBTY Global, Inc.",1,,
2,Jan 5 2023 3:01PM,254397,21,"NBTY Global, Inc.",1,,
3,Jan 5 2023 2:59PM,254396,21,"NBTY Global, Inc.",1,,
4,Jan 5 2023 2:58PM,254395,20,"Emersa Waterbox, LLC",1,,
5,Jan 5 2023 2:53PM,254394,21,"NBTY Global, Inc.",1,,
6,Jan 5 2023 2:50PM,254393,22,"NBTY Global, Inc.",1,,
7,Jan 5 2023 2:49PM,254392,10,"Methapharm, Inc.",18,,
8,Jan 5 2023 2:45PM,254391,21,"NBTY Global, Inc.",1,,
9,Jan 5 2023 2:45PM,254389,19,"Emersa Waterbox, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:30AM,254399,10,Eywa Pharma Inc.,1,,
1,Jan 5 2023 3:08PM,254398,21,"NBTY Global, Inc.",1,,
2,Jan 5 2023 3:01PM,254397,21,"NBTY Global, Inc.",1,,
3,Jan 5 2023 2:59PM,254396,21,"NBTY Global, Inc.",1,,
4,Jan 5 2023 2:58PM,254395,20,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:30AM,254399,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Jan 5 2023 3:08PM,254398,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jan 5 2023 3:01PM,254397,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jan 5 2023 2:59PM,254396,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jan 5 2023 2:58PM,254395,20,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:30AM,254399,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Jan 5 2023 3:08PM,254398,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jan 5 2023 3:01PM,254397,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jan 5 2023 2:59PM,254396,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jan 5 2023 2:58PM,254395,20,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4324426,358.0,9.0,0.0
15,762965,4.0,12.0,24.0
19,1526119,59.0,6.0,0.0
20,763079,3.0,0.0,0.0
21,2035017,7.0,1.0,0.0
22,254393,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4324426,358.0,9.0,0.0
1,15,762965,4.0,12.0,24.0
2,19,1526119,59.0,6.0,0.0
3,20,763079,3.0,0.0,0.0
4,21,2035017,7.0,1.0,0.0
5,22,254393,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,358.0,9.0,0.0
1,15,4.0,12.0,24.0
2,19,59.0,6.0,0.0
3,20,3.0,0.0,0.0
4,21,7.0,1.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,358.0
1,15,Released,4.0
2,19,Released,59.0
3,20,Released,3.0
4,21,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,0.0,24.0,0.0,0.0,0.0,0.0
Executing,9.0,12.0,6.0,0.0,1.0,0.0
Released,358.0,4.0,59.0,3.0,7.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,0.0,24.0,0.0,0.0,0.0,0.0
1,Executing,9.0,12.0,6.0,0.0,1.0,0.0
2,Released,358.0,4.0,59.0,3.0,7.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,0.0,24.0,0.0,0.0,0.0,0.0
1,Executing,9.0,12.0,6.0,0.0,1.0,0.0
2,Released,358.0,4.0,59.0,3.0,7.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()